<!-- PROJECT LOGO -->
<br />
<div align="center">
  <a>
    <img src="https://res.cloudinary.com/dek4evg4t/image/upload/v1729273000/Group_4.png" alt="Logo" width="30%">
  </a>
</div>

### 🖹 Descripción:
Este Proyecto tiene como objetivo aplicar redes neuronales convolucionales (CNN) para realizar una clasificación multiclase de imágenes mediante aprendizaje supervisado. Utilizando el [Covid-19 Image Dataset de Kaggle](https://www.kaggle.com/datasets/pranavraikokte/covid19-image-dataset), que contiene imágenes de rayos X clasificadas en tres categorías (Covid-19, Normal, Neumonía), en este proyecto se desarrollarán clasificadores capaces de diagnosticar enfermedades pulmonares. El proyecto también explora el uso de PyTorch para el desarrollo de modelos de Machine Learning y herramientas de monitoreo, como Weights and Biases, para el seguimiento en tiempo real del proceso de entrenamiento.

### ✍️ Autores:
* Angelo Ortiz Vega - [@angelortizv](https://github.com/angelortizv)
* Alejandro Campos Abarca - [@MajinLoop](https://github.com/MajinLoop)

### 📅 Fecha:
20 de octubre de 2024

### 📝 Notas:
Este es el segundo proyecto del curso IC6200 - Inteligencia Artificial. En este notebook, titulado "Covid-19 Classification", se profundiza en técnicas de data augmentation, preprocesamiento de imágenes con filtros, y fine-tuning de modelos CNN como VGG16 para mejorar la capacidad de generalización de las redes neuronales convolucionales.

### Otras notas:
Asegurarse de contar con Python y las siguientes bibliotecas instaladas: torch, torchvision, cv2, numpy, matplotlib, Pillow.

#  1. Importación de librerías

In [22]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torchvision.models import ResNet50_Weights
from torch.utils.data import DataLoader
from torchsummary import summary

from torchviz import make_dot
os.environ["PATH"] += r";C:\Program Files\Graphviz\bin"


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
import wandb
import cv2 
from PIL import Image
import utils as u

#  2. Configuraciones Iniciales

In [5]:
TRAIN_DATA_PATH = 'data/Covid19-dataset/train'
TEST_DATA_PATH = 'data/Covid19-dataset/test'

## 2.1. Verificación de estado de CUDA

In [6]:
u.check_cuda_info()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CUDA available: True
CUDA version: 12.1
Number of GPUs: 1
Current GPU: 0


## 2.2. Configuración de Weights & Biases

In [7]:
wandb.init(project="CovMedNet_modelo_a", entity="angelortizv-tecnologico-de-costa-rica") 
wandb.config = {
    "learning_rate": 0.001,
    "batch_size": 32,
    "epochs": 10
}

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: angelortizv (angelortizv-tecnologico-de-costa-rica). Use `wandb login --relogin` to force relogin


#  3. Preprocesamiento y data augmentation

## 3.1. Funciones de preprocesamiento

In [8]:
def apply_bilateral_filter(image):
    image = np.array(image)
    filtered_image = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)
    return Image.fromarray(filtered_image)

def apply_canny_edge_filter(image):
    image = np.array(image)
    edges = cv2.Canny(image, 100, 200)
    return Image.fromarray(edges)

In [9]:
"""data_transforms = {
    'raw': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
    'bilateral': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Lambda(lambda x: cv2.bilateralFilter(np.array(x).astype(np.float32), 9, 75, 75)),
        transforms.ToTensor()
    ]),
    'canny': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Lambda(lambda x: cv2.Canny(np.array(x), 100, 200)),
        transforms.ToTensor()
    ])
}"""

data_transforms = {
    'raw': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'bilateral': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Lambda(apply_bilateral_filter),  # Aplicar Bilateral Filter
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'canny': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Lambda(apply_canny_edge_filter),  # Aplicar Canny Edge Filter
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
}

#  4. Carga de datos

In [10]:
data_dir = "data/Covid19-dataset"

In [11]:
image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/train",
                                          transform=data_transforms[x])
                  for x in ['raw', 'bilateral', 'canny']}

test_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/test",
                                         transform=data_transforms[x])
                 for x in ['raw', 'bilateral', 'canny']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True)
               for x in ['raw', 'bilateral', 'canny']}

test_dataloaders = {x: DataLoader(test_datasets[x], batch_size=32, shuffle=False)
                    for x in ['raw', 'bilateral', 'canny']}

#  5. Selección del Modelo A - ResNet50

Razones de elección Resnet

- **Arquitectura Profunda**: ResNet50 tiene 50 capas, lo que le permite aprender representaciones de alto nivel y características complejas sin sufrir el problema del desvanecimiento del gradiente.
- **Conexiones Residuales**: Estas conexiones permiten que la red aprenda funciones de identidad, facilitando el entrenamiento de redes más profundas. Las conexiones residuales permiten que la información fluya más fácilmente a través de la red, lo que mejora la estabilidad del entrenamiento.
- **Rendimiento Sólido**: Ha demostrado ser altamente efectiva en competiciones de clasificación de imágenes, como ImageNet, donde ha logrado clasificaciones superiores.
- **Transfer Learning**: Utiliza pesos preentrenados de ImageNet, lo que ahorra tiempo y recursos, además de proporcionar un buen punto de partida para la clasificación en tareas específicas con conjuntos de datos más pequeños.
- **Flexibilidad**: Se puede personalizar fácilmente para diferentes tareas de clasificación ajustando la capa final sin necesidad de rediseñar toda la red.

In [12]:
weights = ResNet50_Weights.IMAGENET1K_V1  # Cargar los pesos preentrenados
model_a = models.resnet50(weights=weights)
#model_a.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = model_a.fc.in_features
model_a.fc = nn.Linear(num_ftrs, 3)  # 3 clases: Covid-19, Normal, Viral Pneumonia
model_a = model_a.to(device)

El código comienza cargando los pesos preentrenados de **ResNet50** desde el conjunto de datos **ImageNet** utilizando `ResNet50_Weights.IMAGENET1K_V1`, lo que permite que el modelo aproveche el conocimiento adquirido previamente para reconocer características visuales. Luego, se inicializa el modelo ResNet50 con estos pesos mediante `models.resnet50(weights=weights)`. A continuación, se obtiene el número de características de la capa de salida del modelo original a través de `num_ftrs = model_a.fc.in_features`, y se reemplaza la capa final por una nueva capa lineal (`nn.Linear`) que está configurada para clasificar tres clases específicas: **Covid-19**, **Normal** y **Viral Pneumonia**. 

In [13]:
summary(model_a, (3, 224, 224)) 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [14]:
# Definir la transformación para convertir a escala de grises
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convertir a 3 canales (RGB)
    transforms.Resize((224, 224)),  # Redimensionar a 224x224
    transforms.ToTensor()  # Convertir a tensor
])

dummy_image = Image.new('RGB', (224, 224), color='white')  
dummy_input = transform(dummy_image).unsqueeze(0).to(device) 
dummy_input = transform(dummy_image).unsqueeze(0).to(device) 

output = model_a(dummy_input)
dot = make_dot(output, params=dict(list(model_a.named_parameters())))
dot.render("resnet50_architecture", format="png") 

dot.view() 

'resnet50_architecture.pdf'

#  6. Definición de función de pérdida y optimizador

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer_a = optim.Adam(model_a.parameters(), lr=0.001)

In [16]:
#train_model(model_a, {'train': dataloaders['bilateral'], 'val': test_dataloaders['bilateral']}, criterion, optimizer_a)

#  7. Entrenamiento con los diferentes datasets y registro en W&B

In [23]:
def train_model_with_wandb(model, dataloaders, criterion, optimizer, num_epochs=20, dataset_name="raw"):
    all_preds = []  
    all_labels = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Fases de entrenamiento y validación
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Modo de entrenamiento
            else:
                model.eval()   # Modo de validación

            running_loss = 0.0
            running_corrects = 0

            # Iteramos sobre los datos
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zerar gradientes
                optimizer.zero_grad()

                # Hacer forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Solo hacer backward en la fase de entrenamiento
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Actualizamos loss y accuracy
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Registro de métricas en W&B
            if phase == 'val':
                wandb.log({
                    f"{dataset_name}_val_loss": epoch_loss,
                    f"{dataset_name}_val_acc": epoch_acc
                })
            else:
                wandb.log({
                    f"{dataset_name}_train_loss": epoch_loss,
                    f"{dataset_name}_train_acc": epoch_acc
                })

        if all_labels:  # Asegurarse de que hay etiquetas verdaderas
        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(10, 7))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=[f'Class {i}' for i in range(cm.shape[1])],
                    yticklabels=[f'Class {i}' for i in range(cm.shape[1])])
        plt.title(f'Confusion Matrix for {dataset_name}')
        plt.xlabel('Predicted Labels')
        plt.ylabel('True Labels')
        plt.show()

In [24]:
wandb.watch(model_a, log="all")

## 7.1. Entrenar modelo con dataset crudo (raw)

In [ ]:
train_model_with_wandb(model_a, {'train': dataloaders['raw'], 'val': test_dataloaders['raw']}, criterion, optimizer_a, dataset_name="raw")

## 7.2. Entrenar modelo con dataset filtrado bilateral

In [114]:
train_model_with_wandb(model_a, {'train': dataloaders['bilateral'], 'val': test_dataloaders['bilateral']}, criterion, optimizer_a, dataset_name="bilateral")

Epoch 1/25
----------
train Loss: 0.2011 Acc: 0.9442
val Loss: 0.5030 Acc: 0.8485
Epoch 2/25
----------
train Loss: 0.1194 Acc: 0.9562
val Loss: 2.1469 Acc: 0.6970
Epoch 3/25
----------
train Loss: 0.0636 Acc: 0.9841
val Loss: 0.9055 Acc: 0.8939
Epoch 4/25
----------
train Loss: 0.0335 Acc: 0.9841
val Loss: 0.0273 Acc: 0.9848
Epoch 5/25
----------
train Loss: 0.0760 Acc: 0.9681
val Loss: 1.0801 Acc: 0.8182
Epoch 6/25
----------
train Loss: 0.0947 Acc: 0.9641
val Loss: 0.1294 Acc: 0.9242
Epoch 7/25
----------
train Loss: 0.0351 Acc: 0.9920
val Loss: 0.4144 Acc: 0.9091
Epoch 8/25
----------
train Loss: 0.0166 Acc: 0.9960
val Loss: 1.1925 Acc: 0.7727
Epoch 9/25
----------
train Loss: 0.0291 Acc: 0.9880
val Loss: 0.1501 Acc: 0.9242
Epoch 10/25
----------
train Loss: 0.0587 Acc: 0.9801
val Loss: 0.5010 Acc: 0.9242
Epoch 11/25
----------
train Loss: 0.0573 Acc: 0.9761
val Loss: 0.2355 Acc: 0.9242
Epoch 12/25
----------
train Loss: 0.0409 Acc: 0.9880
val Loss: 0.0062 Acc: 1.0000
Epoch 13/25
-

## 7.3. Entrenar modelo con dataset con filtro Canny

In [115]:
model_a.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model_a = model_a.to(device)

train_model_with_wandb(model_a, {'train': dataloaders['canny'], 'val': test_dataloaders['canny']}, criterion, optimizer_a, dataset_name="canny")

Epoch 1/25
----------
train Loss: 1.1291 Acc: 0.7530
val Loss: 5.4246 Acc: 0.3485
Epoch 2/25
----------
train Loss: 0.3640 Acc: 0.8566
val Loss: 5.6982 Acc: 0.3030
Epoch 3/25
----------
train Loss: 0.3675 Acc: 0.8207
val Loss: 5.2198 Acc: 0.3636
Epoch 4/25
----------
train Loss: 0.2993 Acc: 0.8845
val Loss: 14.2764 Acc: 0.3182
Epoch 5/25
----------
train Loss: 0.2379 Acc: 0.9283
val Loss: 1.1999 Acc: 0.7273
Epoch 6/25
----------
train Loss: 0.1266 Acc: 0.9522
val Loss: 0.4790 Acc: 0.8333
Epoch 7/25
----------
train Loss: 0.1246 Acc: 0.9602
val Loss: 1.1682 Acc: 0.6970
Epoch 8/25
----------
train Loss: 0.1029 Acc: 0.9602
val Loss: 0.3262 Acc: 0.8485
Epoch 9/25
----------
train Loss: 0.0927 Acc: 0.9641
val Loss: 0.6877 Acc: 0.7879
Epoch 10/25
----------
train Loss: 0.0945 Acc: 0.9641
val Loss: 0.8625 Acc: 0.7121
Epoch 11/25
----------
train Loss: 0.0273 Acc: 0.9920
val Loss: 1.5759 Acc: 0.5758
Epoch 12/25
----------
train Loss: 0.0325 Acc: 0.9960
val Loss: 0.4374 Acc: 0.8788
Epoch 13/25


# 8. Exportar resultados

In [123]:
torch.save(model_a.state_dict(), 'model_a.pth')

In [124]:
wandb.finish()

bilateral_train_acc,▁▃▆▆▄▄▇█▇▆▅▇▇████▇▆▇█▅█▆▇
bilateral_train_loss,█▅▃▂▄▄▂▁▂▃▃▂▂▁▁▁▂▂▂▂▁▂▁▂▂
bilateral_val_acc,▅▁▆█▄▆▆▃▆▆▆█▇█▇▇█▁▆▇▇▄▅▇▇
bilateral_val_loss,▃█▄▁▄▁▂▅▁▃▂▁▁▁▁▁▁█▂▂▂▄▂▁▁
canny_train_acc,▁▄▃▅▆▇▇▇▇▇███████████████
canny_train_loss,█▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
canny_val_acc,▂▁▂▁▆▇▆▇▇▆▄█▇███▇▇▇▇▇████
canny_val_loss,▄▄▃█▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
raw_train_acc,▁▃▃▄▆▅▇▇▇▆▇▇▇█████▇▇▇███▇
raw_train_loss,▇█▅▄▄▃▂▂▁▂▂▂▂▁▁▁▁▁▁▂▂▁▁▁▂
raw_val_acc,▄▁▅▆██▇▇█▆▂▃▆▇▇▆▅▇▆▆█▇▇█▇
